In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set_style('darkgrid')

from PIL import Image, ImageDraw
import tensorflow as tf

import os
import ast  ## Change str -> list.
import sys
import time

import warnings
warnings.filterwarnings('ignore')

import greatbarrierreef

In [ ]:
df_train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df_train['img_path'] = os.path.join('../input/tensorflow-great-barrier-reef/train_images')+"/video_"+df_train.video_id.astype(str)+"/"+df_train.video_frame.astype(str)+".jpg"
df_train.head()

In [ ]:
df_train['annotations'] = df_train['annotations'].apply(lambda x: ast.literal_eval(x))
df_train['Number_bbox'] = df_train['annotations'].apply(lambda x:len(x)) 
df_train = df_train.query("Number_bbox>0")
df_train.head()

In [ ]:
def bbox_area(anotations):
    area_list = []
    for anno in anotations:
#         print(anno)
        area = anno["width"] * anno["height"]
        area_list.append(area)
    return area_list

df_train["bbox_area"] = df_train["annotations"].apply(bbox_area)
df_train["max_area"] = df_train["bbox_area"].apply(lambda x : max(x))
df_train["min_area"] = df_train["bbox_area"].apply(lambda x : min(x))
df_train.head()



In [ ]:
def img_viz(df_train, id):
    image = df_train['img_path'][id]
    img = Image.open(image)
    
    for box in df_train['annotations'][id]:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)
    display(img)

In [ ]:
df_train.sort_values("max_area", ascending=False).head()

In [ ]:
img_viz(df_train, 7336)

In [ ]:
df_train.sort_values("min_area").head()

In [ ]:
img_viz(df_train, 9233)

In [ ]:
img_viz(df_train, 5433)

In [ ]:
import collections
import tqdm


all_boxes_area = []

for index, row in df_train.iterrows():
    all_boxes_area.extend(row['bbox_area'])
        

# c = collections.Counter(all_boxes_area)
# fig = plt.figure(figsize=(24, 8))
# plt.hist(x=all_boxes_area, log=True)    

ax_h_w = (2,1)
# ratio = 6
fig1 = plt.figure(figsize=(24, 18))
ax1 = fig1.add_subplot(*ax_h_w, 1)
ax1.hist(x=all_boxes_area, bins=50)
ax2 = fig1.add_subplot(*ax_h_w, 2)
ax2.hist(x=all_boxes_area, bins=50, log=True)
plt.show()

In [ ]:
pd.DataFrame(pd.Series(np.array(all_boxes_area).ravel()).describe())